In [1]:
# Import dependencies
import pandas as pd 

In [9]:
behaviorialrisk_df.columns

Index(['Year', 'LocationAbbr', 'LocationDesc', 'Datasource', 'PriorityArea1',
       'PriorityArea2', 'PriorityArea3', 'PriorityArea4', 'Category', 'Topic',
       'Indicator', 'Break_Out_Category', 'Break_out', 'Data_Value_Type',
       'Data_Value_Unit', 'Data_Value', 'Data_Value_Footnote_Symbol',
       'Data_Value_Footnote', 'Confidence_Limit_Low', 'Confidence_Limit_High',
       'CategoryID', 'TopicID', 'IndicatorID', 'BreakoutCategoryID',
       'BreakOutID', 'LocationID', 'GeoLocation'],
      dtype='object')

In [47]:
# Reference to CSV and reading CSV into Pandas DataFrame
csv_path = "Resources/HDMortality.csv"
hdmortality_df = pd.read_csv(csv_path)
hdmortality_df.head(2)

Year LocationAbbr       LocationDesc GeographicLevel DataSource  \
0  2016           AZ  Santa Cruz County          County       NVSS   
1  2016           CA     Alameda County          County       NVSS   

                     Class                    Topic Data_Value  \
0  Cardiovascular Diseases  Heart Disease Mortality      178.8   
1  Cardiovascular Diseases  Heart Disease Mortality        175   

          Data_Value_Unit                                    Data_Value_Type  \
0  per 100,000 population  Age-adjusted, Spatially Smoothed, 3-year Avera...   
1  per 100,000 population  Age-adjusted, Spatially Smoothed, 3-year Avera...   

     ...    Stratification1 StratificationCategory2  \
0    ...             Female          Race/Ethnicity   
1    ...               Male          Race/Ethnicity   

              Stratification2 TopicID LocationID      Y_lat       X_lon  \
0                    Hispanic      T2       4023  31.529969 -110.846130   
1  Asian and Pacific Islander      T2       6001  37.642754 -121.877537   

                        Georeference Column  States  Counties  
0         POINT (-110.8461304 31.529969244)     7.0         9  
1  POINT (-121.87753659999998 37.642753818)     8.0     1,141  

[2 rows x 23 columns]

In [48]:
hdmortality_df.columns

Index(['Year', 'LocationAbbr', 'LocationDesc', 'GeographicLevel', 'DataSource',
       'Class', 'Topic', 'Data_Value', 'Data_Value_Unit', 'Data_Value_Type',
       'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'StratificationCategory1', 'Stratification1', 'StratificationCategory2',
       'Stratification2', 'TopicID', 'LocationID', 'Y_lat', 'X_lon',
       'Georeference Column', 'States', 'Counties'],
      dtype='object')

In [9]:
# Reference to CSV and reading CSV into Pandas DataFrame
csv_path = "Resources/NHIS_heart_survey.csv"
nhis_df = pd.read_csv(csv_path)
nhis_df.head(2)

Year LocationAbbr LocationDesc DataSource PriorityArea1 PriorityArea2  \
0  2009          MDW      Midwest       NHIS          None          ABCS   
1  2009          SOU        South       NHIS          None          None   

         PriorityArea3 PriorityArea4      Category                Topic  \
0                 None          None  Risk Factors              Smoking   
1  Healthy People 2020          None  Risk Factors  Physical Inactivity   

      ...     Break_Out_Category           Break_Out CategoryId  TopicId  \
0     ...                 Gender                Male         C2      T15   
1     ...                   Race  Non-Hispanic Black         C2      T14   

   IndicatorID Data_Value_TypeID BreakOutCategoryId  BreakOutId  LocationID  \
0        NS010           AgeStdz              BOC02       GEN01        9002   
1        NS011             Crude              BOC04       RAC02        9003   

  GeoLocation  
0         NaN  
1         NaN  

[2 rows x 29 columns]

In [10]:
nhis_df.columns

Index(['Year', 'LocationAbbr', 'LocationDesc', 'DataSource', 'PriorityArea1',
       'PriorityArea2', 'PriorityArea3', 'PriorityArea4', 'Category', 'Topic',
       'Indicator', 'Data_Value_Type', 'Data_Value_Unit', 'Data_Value',
       'Data_Value_Alt', 'Data_Value_Footnote_Symbol', 'Data_Value_Footnote',
       'LowConfidenceLimit', 'HighConfidenceLimit', 'Break_Out_Category',
       'Break_Out', 'CategoryId', 'TopicId', 'IndicatorID',
       'Data_Value_TypeID', 'BreakOutCategoryId', 'BreakOutId', 'LocationID',
       'GeoLocation'],
      dtype='object')

In [2]:
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkFiles
from pyspark.sql import DataFrameReader
from pyspark.sql.functions import to_timestamp
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import *

In [3]:
spark = SparkSession \
    .builder \
    .appName('pyspark_demo_app') \
    .config('spark.driver.extraClassPath',
            'postgresql-42.2.9') \
    .master("local[*]") \
    .getOrCreate()

In [15]:
cardio_train_sdf = spark.read.format('csv').options(header='true', inferSchema='true', sep=";").load("Resources/cardio_train.csv")
cardio_train_sdf.show(3)

+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
| id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|  0|18393|     2|   168|  62.0|  110|   80|          1|   1|    0|   0|     1|     0|
|  1|20228|     1|   156|  85.0|  140|   90|          3|   1|    0|   0|     1|     1|
|  2|18857|     1|   165|  64.0|  130|   70|          3|   1|    0|   0|     0|     1|
+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
only showing top 3 rows



In [5]:
nhis_sdf = spark.read.format('csv').options(header='true', inferSchema='true').load("Resources/NHIS_heart_survey.csv")
nhis_sdf.show(3)

+----+------------+------------+----------+--------------+-------------+-------------------+-------------+--------------------+-------------------+--------------------+----------------+---------------+----------+--------------+--------------------------+-------------------+------------------+-------------------+------------------+------------------+----------+-------+-----------+-----------------+------------------+----------+----------+-----------+
|Year|LocationAbbr|LocationDesc|DataSource| PriorityArea1|PriorityArea2|      PriorityArea3|PriorityArea4|            Category|              Topic|           Indicator| Data_Value_Type|Data_Value_Unit|Data_Value|Data_Value_Alt|Data_Value_Footnote_Symbol|Data_Value_Footnote|LowConfidenceLimit|HighConfidenceLimit|Break_Out_Category|         Break_Out|CategoryId|TopicId|IndicatorID|Data_Value_TypeID|BreakOutCategoryId|BreakOutId|LocationID|GeoLocation|
+----+------------+------------+----------+--------------+-------------+-------------------+

In [4]:
entireDF4 = spark.read.jdbc(url=jdbc_url, table = 'heart_cardio_train', properties=dbConfig)
entireDF4.printSchema()
entireDF4.limit(5).show()

NameError: name 'jdbc_url' is not defined

In [ ]:
entireDF4 = spark.read.jdbc(url=jdbc_url, table = 'heart_nhis', properties=dbConfig)
entireDF4.printSchema()
entireDF4.limit(5).show()

In [ ]:
behavior_sdf = spark.read.format('csv').options(header='true', inferSchema='true').load("Resources/Behavioral_Risk.csv")
behavior_sdf.show(3)

In [ ]:
mortality_sdf = spark.read.format('csv').options(header='true', inferSchema='true').load("Resources/HDMortality.csv")
mortality_sdf.show(3)

In [51]:
cleveland_sdf = spark.read.format('csv').options(header='false', inferSchema='true').load("Resources/ml_cleveland_data.csv")
cleveland_sdf.show(3)

+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
| _c0|_c1|_c2|  _c3|  _c4|_c5|_c6|  _c7|_c8|_c9|_c10|_c11|_c12|_c13|
+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
|63.0|1.0|1.0|145.0|233.0|1.0|2.0|150.0|0.0|2.3| 3.0| 0.0| 6.0|   0|
|67.0|1.0|4.0|160.0|286.0|0.0|2.0|108.0|1.0|1.5| 2.0| 3.0| 3.0|   2|
|67.0|1.0|4.0|120.0|229.0|0.0|2.0|129.0|1.0|2.6| 2.0| 2.0| 7.0|   1|
+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
only showing top 3 rows



In [5]:
# tips on jdbc ... https://sparkour.urizone.net/recipes/using-jdbc/#03
# Load properties from file
import json
with open("Resources/db-properties.json") as propertyFile:
    properties = json.load(propertyFile)

dbMode = "overwrite"
jdbc_url= properties["jdbcUrl"]
dbConfig = {"user": properties["user"], 
          "password": properties["password"], 
          "driver": properties["driver"]
         }


In [6]:
entireDF4 = spark.read.jdbc(url=jdbc_url, table = 'heart_cardio_train', properties=dbConfig)
entireDF4.printSchema()
entireDF4.limit(5).show()

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- ap_hi: integer (nullable = true)
 |-- ap_lo: integer (nullable = true)
 |-- cholesterol: integer (nullable = true)
 |-- gluc: integer (nullable = true)
 |-- smoke: integer (nullable = true)
 |-- alco: integer (nullable = true)
 |-- active: integer (nullable = true)
 |-- cardio: integer (nullable = true)

+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
| id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|  0|18393|     2|   168|  62.0|  110|   80|          1|   1|    0|   0|     1|     0|
|  1|20228|     1|   156|  85.0|  140|   90|          3|   1|    0|   0|     1|     1|
|  2|18857|     1|   165|  64.0|  130|   70|          3|   1| 

In [8]:
# convert spark df to pandas df for pandas manipulation
heart_cardio = entireDF4.select("*").toPandas()

In [9]:
heart_cardio.head()

id    age  gender  height  weight  ap_hi  ap_lo  cholesterol  gluc  smoke  \
0   0  18393       2     168    62.0    110     80            1     1      0   
1   1  20228       1     156    85.0    140     90            3     1      0   
2   2  18857       1     165    64.0    130     70            3     1      0   
3   3  17623       2     169    82.0    150    100            1     1      0   
4   4  17474       1     156    56.0    100     60            1     1      0   

   alco  active  cardio  
0     0       1       0  
1     0       1       1  
2     0       0       1  
3     0       1       1  
4     0       0       0

In [11]:
heart_cardio.describe()

id           age        gender        height        weight  \
count  70000.000000  70000.000000  70000.000000  70000.000000  70000.000000   
mean   49972.419900  19468.865814      1.349571    164.359229     74.205690   
std    28851.302323   2467.251667      0.476838      8.210126     14.395757   
min        0.000000  10798.000000      1.000000     55.000000     10.000000   
25%    25006.750000  17664.000000      1.000000    159.000000     65.000000   
50%    50001.500000  19703.000000      1.000000    165.000000     72.000000   
75%    74889.250000  21327.000000      2.000000    170.000000     82.000000   
max    99999.000000  23713.000000      2.000000    250.000000    200.000000   

              ap_hi         ap_lo   cholesterol          gluc         smoke  \
count  70000.000000  70000.000000  70000.000000  70000.000000  70000.000000   
mean     128.817286     96.630414      1.366871      1.226457      0.088129   
std      154.011419    188.472530      0.680250      0.572270      0.283484   
min     -150.000000    -70.000000      1.000000      1.000000      0.000000   
25%      120.000000     80.000000      1.000000      1.000000      0.000000   
50%      120.000000     80.000000      1.000000      1.000000      0.000000   
75%      140.000000     90.000000      2.000000      1.000000      0.000000   
max    16020.000000  11000.000000      3.000000      3.000000      1.000000   

               alco        active        cardio  
count  70000.000000  70000.000000  70000.000000  
mean       0.053771      0.803729      0.499700  
std        0.225568      0.397179      0.500003  
min        0.000000      0.000000      0.000000  
25%        0.000000      1.000000      0.000000  
50%        0.000000      1.000000      0.000000  
75%        0.000000      1.000000      1.000000  
max        1.000000      1.000000      1.000000

In [17]:
# Append DataFrame to heart_behavior_risk table
cardio_train_sdf.write.jdbc(url=jdbc_url, table = 'heart_cardio_train', mode=dbMode, properties=dbConfig)

In [13]:
# Append DataFrame to heart_behavior_risk table
nhis_sdf.write.jdbc(url=jdbc_url, table = 'heart_nhis', mode=dbMode, properties=dbConfig)

In [45]:
# Append DataFrame to heart_behavior_risk table
behavior_sdf.write.jdbc(url=jdbc_url, table = 'heart_behavior_risk', mode=dbMode, properties=dbConfig)

In [49]:
# Append DataFrame to heart_mortality table
mortality_sdf.write.jdbc(url=jdbc_url, table = 'heart_mortality', mode=dbMode, properties=dbConfig)

In [53]:
# Append DataFrame to heart_mortality table
cleveland_sdf.write.jdbc(url=jdbc_url, table = 'heart_ml_cleveland', mode=dbMode, properties=dbConfig)

In [18]:
entireDF5 = spark.read.jdbc(url=jdbc_url, table = 'heart_cardio_train', properties=dbConfig)
entireDF5.printSchema()
entireDF5.limit(5).show()

root
 |-- id: integer (nullable = true)
 |-- age: integer (nullable = true)
 |-- gender: integer (nullable = true)
 |-- height: integer (nullable = true)
 |-- weight: double (nullable = true)
 |-- ap_hi: integer (nullable = true)
 |-- ap_lo: integer (nullable = true)
 |-- cholesterol: integer (nullable = true)
 |-- gluc: integer (nullable = true)
 |-- smoke: integer (nullable = true)
 |-- alco: integer (nullable = true)
 |-- active: integer (nullable = true)
 |-- cardio: integer (nullable = true)

+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
| id|  age|gender|height|weight|ap_hi|ap_lo|cholesterol|gluc|smoke|alco|active|cardio|
+---+-----+------+------+------+-----+-----+-----------+----+-----+----+------+------+
|  0|18393|     2|   168|  62.0|  110|   80|          1|   1|    0|   0|     1|     0|
|  1|20228|     1|   156|  85.0|  140|   90|          3|   1|    0|   0|     1|     1|
|  2|18857|     1|   165|  64.0|  130|   70|          3|   1| 

In [14]:
entireDF4 = spark.read.jdbc(url=jdbc_url, table = 'heart_nhis', properties=dbConfig)
entireDF4.printSchema()
entireDF4.limit(5).show()

root
 |-- Year: integer (nullable = true)
 |-- LocationAbbr: string (nullable = true)
 |-- LocationDesc: string (nullable = true)
 |-- DataSource: string (nullable = true)
 |-- PriorityArea1: string (nullable = true)
 |-- PriorityArea2: string (nullable = true)
 |-- PriorityArea3: string (nullable = true)
 |-- PriorityArea4: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Data_Value_Type: string (nullable = true)
 |-- Data_Value_Unit: string (nullable = true)
 |-- Data_Value: double (nullable = true)
 |-- Data_Value_Alt: double (nullable = true)
 |-- Data_Value_Footnote_Symbol: string (nullable = true)
 |-- Data_Value_Footnote: string (nullable = true)
 |-- LowConfidenceLimit: double (nullable = true)
 |-- HighConfidenceLimit: double (nullable = true)
 |-- Break_Out_Category: string (nullable = true)
 |-- Break_Out: string (nullable = true)
 |-- CategoryId: string (nullable = true)
 |-- 

In [54]:
entireDF3 = spark.read.jdbc(url=jdbc_url, table = 'heart_ml_cleveland', properties=dbConfig)
entireDF3.printSchema()
entireDF3.limit(5).show()

root
 |-- _c0: double (nullable = true)
 |-- _c1: double (nullable = true)
 |-- _c2: double (nullable = true)
 |-- _c3: double (nullable = true)
 |-- _c4: double (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: double (nullable = true)
 |-- _c7: double (nullable = true)
 |-- _c8: double (nullable = true)
 |-- _c9: double (nullable = true)
 |-- _c10: double (nullable = true)
 |-- _c11: string (nullable = true)
 |-- _c12: string (nullable = true)
 |-- _c13: integer (nullable = true)

+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
| _c0|_c1|_c2|  _c3|  _c4|_c5|_c6|  _c7|_c8|_c9|_c10|_c11|_c12|_c13|
+----+---+---+-----+-----+---+---+-----+---+---+----+----+----+----+
|63.0|1.0|1.0|145.0|233.0|1.0|2.0|150.0|0.0|2.3| 3.0| 0.0| 6.0|   0|
|67.0|1.0|4.0|160.0|286.0|0.0|2.0|108.0|1.0|1.5| 2.0| 3.0| 3.0|   2|
|67.0|1.0|4.0|120.0|229.0|0.0|2.0|129.0|1.0|2.6| 2.0| 2.0| 7.0|   1|
|37.0|1.0|3.0|130.0|250.0|0.0|0.0|187.0|0.0|3.5| 3.0| 0.0| 3.0|   0|
|41.0|0.0|2.0|13

In [50]:
entireDF2 = spark.read.jdbc(url=jdbc_url, table = 'heart_mortality', properties=dbConfig)
entireDF2.printSchema()
entireDF2.limit(5).show()

root
 |-- Year: integer (nullable = true)
 |-- LocationAbbr: string (nullable = true)
 |-- LocationDesc: string (nullable = true)
 |-- GeographicLevel: string (nullable = true)
 |-- DataSource: string (nullable = true)
 |-- Class: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- Data_Value: string (nullable = true)
 |-- Data_Value_Unit: string (nullable = true)
 |-- Data_Value_Type: string (nullable = true)
 |-- Data_Value_Footnote_Symbol: string (nullable = true)
 |-- Data_Value_Footnote: string (nullable = true)
 |-- StratificationCategory1: string (nullable = true)
 |-- Stratification1: string (nullable = true)
 |-- StratificationCategory2: string (nullable = true)
 |-- Stratification2: string (nullable = true)
 |-- TopicID: string (nullable = true)
 |-- LocationID: integer (nullable = true)
 |-- Y_lat: double (nullable = true)
 |-- X_lon: double (nullable = true)
 |-- Georeference Column: string (nullable = true)
 |-- States: integer (nullable = true)
 |-- Countie

In [46]:
entireDF = spark.read.jdbc(url=jdbc_url, table = 'heart_behavior_risk', properties=dbConfig)
entireDF.printSchema()
entireDF.limit(5).show()


root
 |-- Year: integer (nullable = true)
 |-- LocationAbbr: string (nullable = true)
 |-- LocationDesc: string (nullable = true)
 |-- Datasource: string (nullable = true)
 |-- PriorityArea1: string (nullable = true)
 |-- PriorityArea2: string (nullable = true)
 |-- PriorityArea3: string (nullable = true)
 |-- PriorityArea4: string (nullable = true)
 |-- Category: string (nullable = true)
 |-- Topic: string (nullable = true)
 |-- Indicator: string (nullable = true)
 |-- Break_Out_Category: string (nullable = true)
 |-- Break_out: string (nullable = true)
 |-- Data_Value_Type: string (nullable = true)
 |-- Data_Value_Unit: string (nullable = true)
 |-- Data_Value: double (nullable = true)
 |-- Data_Value_Footnote_Symbol: string (nullable = true)
 |-- Data_Value_Footnote: string (nullable = true)
 |-- Confidence_Limit_Low: double (nullable = true)
 |-- Confidence_Limit_High: double (nullable = true)
 |-- CategoryID: string (nullable = true)
 |-- TopicID: string (nullable = true)
 |-- Ind

In [15]:
# https://docs.aws.amazon.com/glue/latest/dg/aws-glue-api-crawler-pyspark-extensions-types.html
# from pyspark.sql.types import *
# schema = StructType(
#     [StructField("Year", IntegerType(), True),
#      StructField("LocationAbbr", StringType(), True),
#      StructField("LocationDesc", StringType(), True),
#      StructField("Datasource", StringType(), True),
#      StructField("PriorityArea1", StringType(), True),
#      StructField("PriorityArea2", StringType(), True),
#      StructField("PriorityArea3", StringType(), True),
#      StructField("PriorityArea4", StringType(), True),
#      StructField("Category", StringType(), True),
#      StructField("Topic", StringType(), True),
#      StructField("Indicator", StringType(), True),
#      StructField("Break_Out_Category", StringType(), True),
#      StructField("Break_out", StringType(), True),
#      StructField("Data_Value_Type", StringType(), True),
#      StructField("Data_Value_Unit", StringType(), True),
#      StructField("Data_Value", StringType(), True),
#      StructField("Data_Value_Footnote_Symbol", StringType(), True),
#      StructField("Data_Value_Footnote", StringType(), True),
#      StructField("Confidence_Limit_Low", StringType(), True),
#      StructField("Confidence_Limit_High", StringType(), True),
#      StructField("CategoryID", StringType(), True),
#      StructField("TopicID", StringType(), True),
#      StructField("IndicatorID", StringType(), True),
#      StructField("BreakoutCategoryID", StringType(), True),
#      StructField("BreakOutID", StringType(), True),
#      StructField("LocationID", StringType(), True),
#      StructField("GeoLocation", StringType(), True)])

# pd_df = pd.read_csv("Resources/Behavioral_Risk.csv")
# sp_df = spark.createDataFrame(pd_df, schema=schema)

In [ ]:
# df = spark.read.csv("Resources/user_data.csv")
# df2 = spark.read.csv("Resources/HDMortality.csv")
# df3 = spark.read.csv("Resources/Behavioral_Risk.csv")
# df.show(5)

In [55]:
from pyspark import SparkFiles
# Load in user_data.csv from S3 into a DataFrame
#get the csv from https://data.world/cdc/behavioral-risk-factor-heart
#rename the file to shorter file name
url = "https://heartsense.s3.us-east-2.amazonaws.com/Behavioral_Risk.csv"
spark.sparkContext.addFile(url)

user_data_df = spark.read.option('header', 'true').csv(SparkFiles.get("Behavioral_Risk.csv"), inferSchema=True, sep=',')
user_data_df.show(2)

+----+------------+------------+----------+-------------+-------------+-------------+-------------+--------------------+--------------------+--------------------+------------------+------------------+---------------+---------------+----------+--------------------------+--------------------+--------------------+---------------------+----------+-------+-----------+------------------+----------+----------+--------------------+
|Year|LocationAbbr|LocationDesc|Datasource|PriorityArea1|PriorityArea2|PriorityArea3|PriorityArea4|            Category|               Topic|           Indicator|Break_Out_Category|         Break_out|Data_Value_Type|Data_Value_Unit|Data_Value|Data_Value_Footnote_Symbol| Data_Value_Footnote|Confidence_Limit_Low|Confidence_Limit_High|CategoryID|TopicID|IndicatorID|BreakoutCategoryID|BreakOutID|LocationID|         GeoLocation|
+----+------------+------------+----------+-------------+-------------+-------------+-------------+--------------------+--------------------+---